# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

**Задача** — пороизвети анализ базы данных книг. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных** 

Таблица `books` cодержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors` cодержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers` cодержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings`cодержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews`содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

**Задания**
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Настройка окружения, загрузка и исследование данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:
def table_head(table):
    query = '''
    SELECT * 
    FROM {}
    '''.format(table) 
    
    df = pd.io.sql.read_sql(query, con = engine)
    
    display(df.head())

In [3]:
table_head('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
table_head('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [5]:
table_head('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
table_head('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
table_head('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Расчет, сколько книг вышло после 1 января 2000 года

In [8]:
query = ''' 
SELECT COUNT (*) AS books_count
  FROM books
 WHERE publication_date > '2000-01-01'
 '''
pd.io.sql.read_sql(query, con = engine) 

,books_count
0,819


**Вывод:** после 1 января 2000 года вышло 819 книг.

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [9]:
query = '''
SELECT b.book_id,
       b.title,
       COUNT (DISTINCT  rew.text) AS reviews_count,
       AVG (rat.rating) AS rating_mean
FROM books as b
LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
LEFT JOIN reviews AS rew ON b.book_id = rew.book_id
GROUP BY b.book_id, 
         b.title
ORDER BY reviews_count DESC
            '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,title,reviews_count,rating_mean
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


**Вывод:** книга с наибольшим количеством обзоров - Twilight, получила 7 обзоров со средним рейтингом 3,66 балла.

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
query = ''' 
SELECT 
    p.publisher,
    COUNT (b.title) as published_books
FROM books as b
LEFT JOIN  publishers as p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY published_books DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher,published_books
0,Penguin Books,42


**Вывод:** Издательство Penguin Books выпустило наибольше число книг толще 50 страниц. Всего 42 книги.

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками


In [12]:
query = ''' 
SELECT
    b.author_id,
    authors.author,
    AVG(ratings.rating) as avg_rating_total
FROM books as b
LEFT JOIN authors ON b.author_id = authors.author_id
LEFT JOIN ratings ON ratings.book_id = b.book_id
WHERE b.book_id IN (SELECT book_id 
                        FROM ratings
                        GROUP BY book_id
                        having COUNT(rating) >= 50)

GROUP BY authors.author,
         b.author_id
ORDER BY avg_rating_total DESC
LIMIT 1

'''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author,avg_rating_total
0,236,J.K. Rowling/Mary GrandPré,4.287097


**Вывод:** Автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками)  - J.K. Rowling/Mary GrandPré

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
query = ''' 
SELECT ROUND(AVG(count)) AS avg_count_rewiew  
FROM
     (SELECT COUNT(text)
      FROM reviews
      FULL OUTER JOIN (SELECT username,
                COUNT(rating_id) as cnt
    FROM ratings
    GROUP BY username) AS cnt_rating
ON cnt_rating.username = reviews.username
WHERE cnt > 50
GROUP BY reviews.username) as count;                                  
                                  
'''
pd.io.sql.read_sql(query, con = engine)

,avg_count_rewiew
0,24.0


**Вывод:** в среднем пользователи, поставившие более 50 оценок, оставляют 24 отзыва.

## Общий вывод

В ходе решения заданий получены следующие результаты:
- после 1 января 2000 года вышло 819 книг
- книга с наибольшим количеством обзоров - Twilight, получила 7 обзоров со средним рейтингом 3,66 балла
- Издательство Penguin Books выпустило наибольше число книг толще 50 страниц. Всего 42 книги
- Автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками) - J.K. Rowling/Mary GrandPré
- в среднем пользователи, поставившие более 50 оценок, оставляют 24 отзыва